## Computing CLEVER metric with blackbox CNN model from Keras trained with CIFAR10 dataset

In [1]:
# import neccessary files
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import tensorflow as tf

# art library
from art import metrics
from art.classifiers import KerasClassifier
from art.utils import load_dataset
from ModelSurrogateCopy1 import modelSurrogate

Using TensorFlow backend.


## Preparing dataset and environment
In this file, we will use the CIFAR10 dataset.
We will disable tensorflow 2.0's eager evaluation as ART has not been updated to support eager evaluation.

In [2]:
tf.compat.v1.disable_eager_execution()
# Read CIFAR10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('cifar10'))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]
im_shape = x_train[0].shape

## Creating and preparing the model
For this example, we will use Keras Convolutional Neural Network model (CNN) for evaluation

In [6]:
# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Wrapping a blackbox classifier
This creates a classifier wrapper for training of model using ART library

In [3]:
# Create classifier wrapper
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

NameError: name 'model' is not defined

## Creating Surrogate model for CLEVER evaluation
as the testing environment is a blackbox model, we will create a surrogate model using prior-based RGF method to replicate the true model, and use this surrogate model as a whitebox to compute CLEVER score.

In [3]:
# create a surrogate model
modelSurr = modelSurrogate(x_train,x_test,y_train,y_test,min_,max_)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Epoch 1/5
157/156 [==============================] - 3s 21ms/step - loss: 2.2995 - accuracy: 0.1210
Epoch 2/5
157/156 [==============================] - 3s 20ms/step - loss: 2.2833 - accuracy: 0.1541
Epoch 3/5
157/156 [==============================] - 3s 20ms/step - loss: 2.2558 - accuracy: 0.1799
Epoch 4/5
157/156 [==============================] - 3s 20ms/step - loss: 2.2270 - accuracy: 0.1921
Epoch 5/5
157/156 [==============================] - 3s 20ms/step - loss: 2.2051 - accuracy: 0.1959


In [4]:
grad = modelSurr.gradient_norm(x_test[:1],y_test[:1])
modelSurr.cos_similarity(grad,x_test[:1],y_test[:1],3,0.6,10)

(2, 2)

In [4]:
modelSurr.clever(x_train[0])

2.578138239174868